# Sampling out-of-equilibrium improves the training of Restricted Boltzmann Machines

### Authors:

Charles K. Fisher; charleskennethfisher@gmail.com
Jonathan R. Walsh; jrwalsh1@gmail.com

### Abstract:

Restricted Boltzmann Machines (RBMs) are a class of neural network that can be used to approximate arbitrary probability distributions. Unlike feed-forward neural networks, RBMs are stochastic, which makes them perform well as trainable generative models.  Training an RBM through stochastic gradient descent is difficult, however, because it is necessary to draw a sample from the model distribution for each gradient evaluation. 

This notebook describes an approach to sampling from an RBM by coupling the distribution to a slowly varying auxiliary variable. The non-equilibrium sampler allows the system to move around energy barriers, which results in better mixing. This simple modification leads to large improvements in training -- especially for RBMs with Gaussian visible units -- with minimal computational cost compared to standard samplingmethods. 

### About this project:

This notebook is an experiment in *open science*. It will provide all of the background information, data sets, and code required for a traditional scientific publication -- but in an open-source format that anybody can access (or even contribute to). 

This project is connected to [paysage](https://github.com/drckf/paysage): a Python library for machine learning with Boltzmann machines. The non-equilibrium sampler discussed in this notebook is implemented in paysage as the class `DrivenSequentialMC`. 

## Introduction



In [2]:
# Examples
import paysage

## Discussion



## Acknowledgements


## References